<a href="https://colab.research.google.com/github/apschlissel/w207-final-project/blob/main/madeline/W207_LoadingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
###################Setup instructions####################################
# root directory contains this file          
# root directory has the kaggle installation 
# training director is root/petfinder-pawpularity-score
# training image directory is root/petfinder-pawpularity-score/train
#########################################################################
import numpy as np
import pandas as pd
import os, sys
import cv2
import matplotlib.pyplot as plt
import sys
from PIL import Image, ImageOps
from pathlib import Path

import tensorflow as tf # tensorflow 2.0
from keras.datasets import mnist
import numpy as np

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
path = "/content/drive/MyDrive/W207/petfinder-pawpularity-score/train.csv"
df = pd.read_csv(path)
df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72


In [30]:
# importing and resizing image data 
csv_path = "/content/drive/MyDrive/W207/petfinder-pawpularity-score/train.csv"

df= pd.read_csv(csv_path)
df['path'] = '/content/drive/MyDrive/W207/petfinder-pawpularity-score/train/'+df['Id'] + '.jpg'

# limiting how many records we're transforming due to crashing issues
df2 = df.head(100)

df2['img'] = df2['path'].apply(lambda x: Image.open(x).convert("RGB"))
df2['resize_img'] = df2['img'].apply(lambda x: x.resize((100, 100)))
df2['img_array'] = df2['resize_img'].apply(lambda x: np.array(x, dtype=float))
df2['img_array'] = df2['img_array']/255

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [41]:
# there aren't scores on the test data 
# so we won't be able to use this at all
csv_path = "/content/drive/MyDrive/W207/petfinder-pawpularity-score/test.csv"

df_test= pd.read_csv(csv_path)
df_test['path'] = '/content/drive/MyDrive/W207/petfinder-pawpularity-score/test/'+df_test['Id'] + '.jpg'

df_test['img'] = df_test['path'].apply(lambda x: Image.open(x).convert("RGB"))
df_test['resize_img'] = df_test['img'].apply(lambda x: x.resize((100, 100)))
df_test['img_array'] = df_test['resize_img'].apply(lambda x: np.array(x, dtype=float))
df_test['img_array'] = df_test['img_array']/255

In [49]:
df_test = df[df.index >100]
df_test = df_test[df_test.index <= 200]

df_test['img'] = df_test['path'].apply(lambda x: Image.open(x).convert("RGB"))
df_test['resize_img'] = df_test['img'].apply(lambda x: x.resize((100, 100)))
df_test['img_array'] = df_test['resize_img'].apply(lambda x: np.array(x, dtype=float))
df_test['img_array'] = df_test['img_array']/255

In [50]:
# just grabbed this from this article https://towardsdatascience.com/image-recognition-with-machine-learning-on-python-convolutional-neural-network-363073020588
# to start playing around with it
# getting error because my data is in array form not Tensor

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten 
from keras.layers import MaxPooling2D, Dropout
input_shape = (100, 100, 1)
num_classes = 100
X_train = df2.img_array
Y_train = df2.Pawpularity

X_test = df_test.img_array
Y_test = df_test.Pawpularity

model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=input_shape))

model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(num_classes, activation='softmax'))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=3)
# evaluate the model
score = model.evaluate(X_test, Y_test, verbose=1)
# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: ignored